The solutions are presented at: https://tube.switch.ch/videos/33053e02#25:43

# Link based ranking

**Preliminaries:** If you want to normalize a vector to L1-norm or L2-norm, use:

In [ ]:
from __future__ import print_function, division
import numpy as np

pr = np.array([1,2,3])
print("L1-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,1)))
print("L2-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,2)))

## Page Rank (Eigen-vector method)
Consider a tiny Web with three pages A, B and C with no inlinks,
and with initial PageRank = 1. Initially, none of the pages link to
any other pages and none link to them. 
Answer the following questions, and calculate the PageRank for
each question.

1. Link page A to page B.
2. Link all pages to each other.
3. Link page A to both B and C, and link pages B and C to A.
4. Use the previous links and add a link from page C to page B.

**Hints:**
We are using the theoretical PageRank computation (without source of rank). See slide "Transition Matrix for Random Walker" in the lecture note. Columns of link matrix are from-vertex, rows of link matrix are to-vertex. We take the eigenvector with the largest eigenvalue.
We only care about final ranking of the probability vector. You can choose the normalization (or not) of your choice).

In [ ]:
def create_Rmatrix(L):
#     R = np.multiply(L, 1 / np.sum(L,axis=0)) # Use this matrix multiplication for faster running time if no column is zero
    X = np.sum(L,axis=0)
    n_nodes = L.shape[0]
    R = np.zeros((n_nodes, n_nodes))
    for i in range(L.shape[0]):
        for j in range(L.shape[1]):
            R[i,j] = L[i,j] / X[0,j] if X[0,j] != 0 else 0
            
#     R = np.multiply(L,R)
    return R

In [ ]:
"""Some time we might want to compute R outside the function to avoid recomputing large matrix"""
def pagerank_eigen(L, R=None):
#   Construct transition probability matrix from L
    if R is None: R = create_Rmatrix(L)
#     Compute eigen-vectors and eigen-values of R
    eigenvalues, eigenvectors = np.linalg.eig(R)
#     Take the eigen-vector with maximum eigven-value
    p = eigenvectors[:,np.argmax(np.absolute(eigenvalues))]
    return (R,p)


In [ ]:
# Test with the question, e.g.
L = np.matrix([
    [0,1,1], 
    [1,0,1], 
    [1,1,0]
])
R,p = pagerank_eigen(L)
print("L={0}\nR={1}\np={2}".format(L,R,p))

1.Link page A to page B

Result:
$
L =
  \begin{bmatrix}
	0 & 0 & 0 \\
1 & 0 & 0 \\
0 & 0 & 0 \\
  \end{bmatrix}
 ,
R =
  \begin{bmatrix}
	0 & 0 & 0 \\
1 & 0 & 0 \\
0 & 0 & 0 \\
  \end{bmatrix}
  ,
  \vec{p} =
  \begin{bmatrix}
	0 \\
1 \\
0 \\
  \end{bmatrix}
$

In [ ]:
L = np.matrix([
    [0,0,0], 
    [1,0,0], 
    [0,0,0]
])

R,p = pagerank_eigen(L)
print("L={0}\nR={1}\np={2}".format(L,R,p))

2.Link all pages to each other

Result:
$
L =
  \begin{bmatrix}
	0 & 1 & 1 \\
1 & 0 & 1 \\
1 & 1 & 0 \\
  \end{bmatrix}
 ,
R =
  \begin{bmatrix}
	0 & \frac{1}{2} & \frac{1}{2} \\
\frac{1}{2} & 0 & \frac{1}{2} \\
\frac{1}{2} & \frac{1}{2} & 0 \\
  \end{bmatrix}
  ,
  \vec{p} =
  \begin{bmatrix}
	0.577 \\
0.577 \\
0.577 \\
  \end{bmatrix}
$

In [ ]:
L = np.matrix([
    [0,1,1], 
    [1,0,1], 
    [1,1,0]
])

R,p = pagerank_eigen(L)
print("L={0}\nR={1}\np={2}".format(L,R,p))

3.Link page A to both B and C, and link pages B and C to A.

Result:
$
L =
  \begin{bmatrix}
	0 & 1 & 1 \\
1 & 0 & 0 \\
1 & 0 & 0 \\
  \end{bmatrix}
 ,
R =
  \begin{bmatrix}
	0 & 1 & 1 \\
\frac{1}{2} & 0 & 0 \\
\frac{1}{2} & 0 & 0 \\
  \end{bmatrix}
  ,
  \vec{p} =
  \begin{bmatrix}
	0.816 \\
0.408 \\
0.408 \\
  \end{bmatrix}
$

In [ ]:
L = np.matrix([
    [0,1,1], 
    [1,0,0], 
    [1,0,0]
])

R,p = pagerank_eigen(L)
print("L={0}\nR={1}\np={2}".format(L,R,p))

4.Use the previous links and add a link from page C to page B

Result:
$
L =
  \begin{bmatrix}
	0 & 1 & 1 \\
1 & 0 & 1 \\
1 & 0 & 0 \\
  \end{bmatrix}
 ,
R =
  \begin{bmatrix}
	0 & 1 & \frac{1}{2} \\
\frac{1}{2} & 0 & \frac{1}{2} \\
\frac{1}{2} & 0 & 0 \\
  \end{bmatrix}
  ,
  \vec{p} =
  \begin{bmatrix}
	0.743 \\
0.557 \\
0.371 \\
  \end{bmatrix}
$

In [ ]:
L = np.matrix([
    [0,1,1], 
    [1,0,1], 
    [1,0,0]
])

R,p = pagerank_eigen(L)
print("L={0}\nR={1}\np={2}".format(L,R,p))

## Page Rank (Iterative method)

The eigen-vector method has some numerical issues (when computing eigen-vector) and not scalable with large datasets.

We will apply the iterative method in the slide "Practical Computation of PageRank" of the lecture.

Dataset for practice: https://snap.stanford.edu/data/ca-GrQc.html. It is available within the same folder of this github.

In [ ]:
def pagerank_iterative(L, R=None):
    if R is None: #We might want to compute R outside this function to avoid recomputing large matrix
        R = np.multiply(L, 1 / np.sum(L,axis=0))
        
    N = R.shape[0]
    e = np.ones(shape=(N,1))
    q = 0.9

    p = e
    delta = 1
    epsilon = 0.001
    i = 0
    while delta > epsilon:
        p_prev = p
        p = np.matmul(q * R, p_prev)
        p = p + (1-q) / N * e
        delta = np.linalg.norm(p-p_prev,1)
        i += 1

    print("Converged after {0} iterations".format(i))
    return R,p

### Construct link matrix from dataset

In [ ]:
n_nodes = 0
nodes_idx = dict() #Since the nodeIDs are not from 0 to N we need to build an index of nodes
nodes = [] #We also want to store nodeIDs to return the result of ranking vector

# Read the nodes
with open("ca-GrQc.txt") as f:
    for line in f:
        if '#' not in line:
            source = int(line.split()[0])
            target = int(line.split()[1])
            if source not in nodes_idx.keys():
                nodes_idx[source] = n_nodes
                nodes.append(source)
                n_nodes += 1
            if target not in nodes_idx.keys():
                nodes_idx[target] = n_nodes
                nodes.append(target)
                n_nodes += 1
print(n_nodes)
print(nodes[:3])

In [ ]:
L = np.zeros((n_nodes, n_nodes))
# Read the edges
with open("ca-GrQc.txt") as f:
    for line in f:
        if "#" not in line:
            source = int(line.split()[0])
            target = int(line.split()[1])
            L[nodes_idx[target], nodes_idx[source]] = 1 #Columns of link matrix are from-vertices
print(L)

### Compute transition probability matrix

In [ ]:
# Here I use matrix multiplication from numpy for faster running time
R = np.multiply(L, 1 / np.sum(L,axis=0))
# R = create_Rmatrix(L) # This is much slower
print(R)



### You will see that eigen-vector method is slow and has some numerical issues


In [ ]:
import time
start_time = time.time()
R,p = pagerank_eigen(L, R)
print("--- %s seconds ---" % (time.time() - start_time))
print("Ranking vector: p={0}".format(p))
# print(eigenvectors)

### Iterative method

In [ ]:
import time
start_time = time.time()
R, p = pagerank_iterative(L,R)
print("--- %s seconds ---" % (time.time() - start_time))
print("Ranking vector: p={0}".format(p[:,0]))

### Extract top-k nodes from the ranking vector

In [ ]:
arr = np.array(p[:,0])
k = 3
k_idx = arr.argsort()[-k:][::-1]
print("Top-{0} nodes: {1}".format(k, np.array(nodes)[k_idx]))
print("Their scores: {0}".format(arr[k_idx]))

## Ranking Methodology (Hard)

1. Give a directed graph, as small as possible, satisfying all the properties mentioned below:

    1. There exists a path from node i to node j for all nodes i,j in the directed
graph. Recall, with this property the jump to an arbitrary node in PageRank
is not required, so that you can set q = 1 (refer lecture slides).

    2. HITS authority ranking and PageRank ranking of the graph nodes are different.

2. Give intuition/methodology on how you constructed such a directed graph with
the properties described in (a).

3. Are there specific graph structures with arbitrarily large instances where PageRank
ranking and HITS authority ranking are the same?

Solutions in pdf..

## Hub and Authority

### a)

Let the adjacency matrix for a graph of four vertices ($n_1$ to $n_4$) be
as follows:

$
A =
  \begin{bmatrix}
	0 & 1 & 1 & 1  \\
	0 & 0 & 1 & 1 \\
	1 & 0 & 0 & 1 \\
	0 & 0 & 0 & 1 \\
  \end{bmatrix}
$

Calculate the authority and hub scores for this graph using the
HITS algorithm with k = 6, and identify the best authority and
hub nodes.

### b)
Apply the HITS algorithm to the dataset: https://snap.stanford.edu/data/ca-GrQc.html

**Hint:** We follow the slide "HITS algorithm" in the lecture. **Denote $x$ as authority vector and $y$ as hub vector**. You can use matrix multiplication for the update steps in the slide "Convergence of HITS". Note that rows of adjacency matrix is from-vertex and columns of adjacency matrix is to-vertex.

In [ ]:
def hits_iterative(A, k = 10):
    N = A.shape[0]
    x0, y0 = 1 / (N*N) * np.ones(N), 1 / (N*N) * np.ones(N) 
    xprev, yprev = x0, y0
    delta1 = delta2 = 1
    epsilon = 0.001 # We can strictly check for convergence rate of HITS algorithm
    l = 0
    while l < k and delta1 > epsilon and delta2 > epsilon:
        y = np.matmul(A, xprev)
        x = np.matmul(np.transpose(A), y) 
        x = x / np.linalg.norm(x,2)
        y = y / np.linalg.norm(y,2)
        delta1 = np.linalg.norm(x-xprev,1)
        delta2 = np.linalg.norm(y-yprev,1)
        xprev = x
        yprev = y
        l += 1
    
    print("Ran a total of {0} iterations with the convergence rate delta1, delta2={1},{2}".format(l, delta1, delta2))
    return xprev, yprev

#### a)

In [ ]:
A=np.array([
    [0, 1, 1, 1], 
    [0, 0, 1, 1], 
    [1, 0, 0, 1],
    [0, 0, 0, 1],
])

x, y = hits_iterative(A, 100)
print("Result using iterative method:\n Authoriy vector x={0}\n Hub vector y={1}".format(x, y))

**Details:**
+ Initialization: 
  
  $x_0 = \frac{1}{4^2}(1,1,1,1) = ( 0.0625,  0.0625,  0.0625,  0.0625)$
  
  $y_0 = \frac{1}{4^2}(1,1,1,1) = ( 0.0625,  0.0625,  0.0625,  0.0625)$
  
+ $k=1$:
  
  $x_1 = \frac{A^t y_0}{||A^t y_0||} = (0.21320072,  0.21320072,  0.42640143,  0.85280287)$
  
  $y_1 = \frac{A x_0}{|| A x_0 ||} = (0.70710678,  0.47140452,  0.47140452,  0.23570226)$
  
+ ...:
  
+ $k=6$:
  
   $x_6 = \frac{A^t y_5}{||A^t y_5||} = (0.16887796,  0.27257494,  0.49774555, 0.80587375)$
  
  $y_6 = \frac{A x_5}{||A x_5||} = (0.65357971,  0.54153747,  0.40815386,  0.33612671)$
  

**Conclusion:**
+ Best authority node: $n_4$. Best hub node: $n_1$.
 
**Check with the theoretical result (convergence condition):**
  
+ $x^*$ is the principal eigenvector (i.e. with largest eigenvalue) of $A^t A$: $(0.16845787,  0.27257056,  0.49801119,  0.80579904)$
  
+ $y^*$ is the principal eigenvector (i.e. with largest eigenvalue) of $A A^t$: $(0.65549599,  0.54215478,  0.4051188,   0.33507008)$

#### b)
We reuse the link matrix $L$ to compute the adjacency matrix $A$ of the dataset

In [ ]:
A = np.transpose(L)
print(A)

Run HITS algorithm:

In [ ]:
x, y = hits_iterative(A, 100)
print("Result using iterative method:\n Authoriy vector x={0}\n Hub vector y={1}".format(x, y))

**You can see that the two authority vector and hub vector are the same. So the network must be an undirected graph**

**Interpret the result:**

In [ ]:
k = 3

arr = np.array(x)
k_idx = arr.argsort()[-k:][::-1]
print("Top-{0} authorities: {1}".format(k, np.array(nodes)[k_idx]))
print("Their scores: {0}".format(arr[k_idx]))

arr = np.array(y)
k_idx = arr.argsort()[-k:][::-1]
print("Top-{0} hubs: {1}".format(k, np.array(nodes)[k_idx]))
print("Their scores: {0}".format(arr[k_idx]))

**We can also use linear algebra property of HITS to compute the result (slide "Convergence of HITS"):**.
  
+ $x^*$ is the principal eigenvector (i.e. with largest eigenvalue) of $A^t A$
+ $y^*$ is the principal eigenvector (i.e. with largest eigenvalue) of $A A^t$

However, the computation will be much slower

In [ ]:
xstar_ev, xstar = np.linalg.eig(np.matmul(np.transpose(A),A))
ystar_ev, ystar = np.linalg.eig(np.matmul(A,np.transpose(A)))
xstar, ystar = xstar[:,np.argmax(np.absolute(xstar_ev))], ystar[:,np.argmax(np.absolute(ystar_ev))]
# ystar = -xstar if all(xstar<0) else xstar
# ystar = -ystar if all(ystar<0) else ystar
print("Result using linear algebra:\n Authoriy vector x={0}\n Hub vector y={1}".format(xstar, ystar))